# Computing with Oscillators: A Speech Demo

* author: Nand Chandravadia

***

This tutorial notebook is organized around three main concepts:

- **Part 1: Speech Data**
- **Part 2: Oscillator Model**
- **Part 3: (State-Space) Analysis**

## Part 1: Speech Data